In [ ]:
import os
import psycopg2
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [12]:
from dotenv import load_dotenv

load_dotenv()
PATH_DATA_G = os.getenv('PATH_DATA_G')
G_FILOSOFI = 'filosofi'

In [11]:
import data_gen

bnb2022, bnb2023, décret = data_gen.tables()

Chargement des données Airbnb...
Chargement des données Airbnb terminé.


In [ ]:
conn = psycopg2.connect(
    dbname="obs",
    user="hab",
    password="34*hab",
    host="S1943",
    port="5432",
)
query = """SELECT * FROM hab.iris_100"""
iris = pd.read_sql_query(query, conn)

In [ ]:
bnb2022['geometry'] = bnb2022.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
bnb2022_gdf = gpd.GeoDataFrame(bnb2022, geometry='geometry', crs="EPSG:4326")
bnb2022_gdf = gpd.sjoin(bnb2022_gdf, iris, how="left", predicate="intersects")

# Sélectionner les colonnes d'intérêt
bnb2022_gdf = bnb2022_gdf[['code_officiel_iris', 'code_officiel_commune', 'Code EPCI', 
                        'Nb jours réservés', 'Revenu annuel €', 'Nb total jours de disponibilité']]

# Agréger les données
bnb2022_agg = bnb2022_gdf.groupby(['code_officiel_iris', 'code_officiel_commune']).agg(
    nombre_reserv_120=('Nb jours réservés', lambda x: (x >= 120).sum()),
    nombre_dispo_120=('Nb total jours de disponibilité', lambda x: (x >= 120).sum()),
    nombre_total=('Nb jours réservés', 'count'),
    jours_reserves=('Nb jours réservés', 'sum'),
    revenu=('Revenu annuel €', 'sum'),
    jours_reserves_120=('Nb jours réservés', lambda x: (bnb2022_gdf['Nb total jours de disponibilité'] >= 120).sum()),
    revenu_120=('Revenu annuel €', lambda x: (bnb2022_gdf['Nb total jours de disponibilité'] >= 120).sum())
).reset_index()

# Convertir en DataFrame classique
bnb2022_agg = pd.DataFrame(bnb2022_agg)

# Convertir les colonnes Latitude et Longitude de bnb2023 en valeurs numériques
bnb2023['Latitude'] = pd.to_numeric(bnb2023['Latitude'], errors='coerce')
bnb2023['Longitude'] = pd.to_numeric(bnb2023['Longitude'], errors='coerce')

# Filtrer les observations avec des valeurs non manquantes
bnb2023 = bnb2023.dropna(subset=['Latitude', 'Longitude'])

# Convertir bnb2023 en GeoDataFrame et effectuer la jointure spatiale avec iris
bnb2023['geometry'] = bnb2023.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
bnb2023_gdf = gpd.GeoDataFrame(bnb2023, geometry='geometry', crs="EPSG:4326")
bnb2023_gdf = gpd.sjoin(bnb2023_gdf, iris, how="left", predicate="intersects")

# Sélectionner les colonnes d'intérêt
bnb2023_gdf = bnb2023_gdf[['code_officiel_iris', 'code_officiel_commune', 
                        'Nb jours réservés', 'Revenu annuel €', 'Nb total jours de disponibilité']]

# Agréger les données
bnb2023_agg = bnb2023_gdf.groupby(['code_officiel_iris', 'code_officiel_commune']).agg(
    nombre_reserv_120=('Nb jours réservés', lambda x: (x >= 120).sum()),
    nombre_dispo_120=('Nb total jours de disponibilité', lambda x: (x >= 120).sum()),
    nombre_total=('Nb jours réservés', 'count'),
    jours_reserves=('Nb jours réservés', 'sum'),
    revenu=('Revenu annuel €', 'sum'),
    jours_reserves_120=('Nb jours réservés', lambda x: (bnb2023_gdf['Nb total jours de disponibilité'] >= 120).sum()),
    revenu_120=('Revenu annuel €', lambda x: (bnb2023_gdf['Nb total jours de disponibilité'] >= 120).sum())
).reset_index()

# Convertir en DataFrame classique
bnb2023_agg = pd.DataFrame(bnb2023_agg)

# Fusionner les tables pour comparer les années 2022 et 2023
table_iris = pd.merge(bnb2022_agg, bnb2023_agg, 
                    on=['code_officiel_iris', 'code_officiel_commune'], 
                    how='outer', suffixes=('_2022', '_2023'))

# Calculer les évolutions et les écarts
table_iris = table_iris.assign(
    evol_bnb_120=(table_iris['nombre_dispo_120_2023'] - table_iris['nombre_dispo_120_2022']) / table_iris['nombre_dispo_120_2022'],
    ecart_bnb_120=table_iris['nombre_dispo_120_2023'] - table_iris['nombre_dispo_120_2022'],
    ecart_bnb_tot=table_iris['nombre_total_2023'] - table_iris['nombre_total_2022'],
    evol_bnb_tot=(table_iris['nombre_total_2023'] - table_iris['nombre_total_2022']) / table_iris['nombre_total_2022'],
    ecart_jours=table_iris['jours_reserves_2023'] - table_iris['jours_reserves_2022'],
    ecart_revenu=table_iris['revenu_2023'] - table_iris['revenu_2022'],
    ecart_revenu_jours=(table_iris['revenu_2023'] / table_iris['jours_reserves_2023']) - (table_iris['revenu_2022'] / table_iris['jours_reserves_2022'])
)


In [13]:
chemin = os.path.join(PATH_DATA_G, G_FILOSOFI, "filosofi_iris_2020.xlsx")
filosofi_iris_2020 = pd.read_excel(chemin)

In [14]:
filosofi_iris_2020

,CODGEO,rev_disp_med
0,340030101,13960.0
1,340030102,16970.0
2,340030103,19330.0
3,340030104,20620.0
4,340030106,18540.0
...,...,...
273,343370102,25500.0
274,343370103,22660.0
275,343440101,19790.0
276,343440102,22270.0


In [15]:
chemin = os.path.join(PATH_DATA_G, G_FILOSOFI, "filosofi_com_2020.xlsx")
filosofi_com_2020 = pd.read_excel(chemin)
filosofi_com_2020

,CODGEO,rev_disp_med,nb_men,nb_pers
0,34001,20270,748,1817
1,34002,20140,551,1331
2,34003,19710,16749,31290
3,34004,17680,115,231
4,34005,22580,112,239
...,...,...,...,...
314,34340,25430,598,1593
315,34341,21850,1534,3954
316,34342,24720,90,215
317,34343,23400,503,1194


In [ ]:
chemin = os.path.join(PATH_DATA_G, G_FILOSOFI, "filosofi_com_2020.xlsx")
filosofi_com_2020 = pd.read_excel(chemin)

# Sélection des IRIS de type "commune"
code_iris_egal_com = iris[iris['type'] == "commune"]

# Convertir en DataFrame classique et retirer la colonne 'geometry' si présente
code_iris_egal_com = code_iris_egal_com.drop(columns=['geometry'], errors='ignore')

# Sélectionner les colonnes d'intérêt
code_iris_egal_com = code_iris_egal_com[['code_officiel_iris', 'code_officiel_commune']]

# Merge des données FiLoSoFi Commune avec les IRIS de type "commune"
filosofi_iris_egal_com_2020 = pd.merge(code_iris_egal_com, 
                                    filosofi_com_2020, 
                                    left_on="code_officiel_commune", 
                                    right_on="CODGEO")

# Concaténer les données FiLoSoFi IRIS et Commune pour les IRIS de type "commune"
filosofi_iris_2020 = pd.concat([filosofi_iris_egal_com_2020, filosofi_iris_2020], ignore_index=True)